In [35]:
import pandas as pd
import numpy as np
import geatpy as ea
from multiprocessing import Pool as ProcessPool
import multiprocessing as mp
from multiprocessing.dummy import Pool as ThreadPool

In [36]:
场地处理能力df = pd.read_excel('../../preprocess/场地处理能力.xlsx')
场地处理能力df.set_index('场地', inplace=True)
线路运输能力df = pd.read_excel('../../preprocess/线路运输能力.xlsx')
rawData = pd.read_excel('../../T1/解题/时间窗口为5-第一问结果LTSM-物流网络历史货量数据.xlsx')
rawDataMelt = pd.melt(pd.merge(rawData.iloc[:, :2], rawData.iloc[:, -31:], left_index=True, right_index=True),
                      id_vars=['场地1', '场地2'], value_name='货量', var_name='日期')
totalEdgeLen = len(rawData)
print(f'总线路数量：{totalEdgeLen}')
print(rawDataMelt.dtypes)
rawDataMelt

总线路数量：1049
场地1     int64
场地2     int64
日期     object
货量      int64
dtype: object


,场地1,场地2,日期,货量
0,1,8,2023-01-01,3
1,2,3,2023-01-01,1
2,2,4,2023-01-01,2
3,2,5,2023-01-01,10
4,2,8,2023-01-01,2
...,...,...,...,...
32514,76,8,2023-01-31,15
32515,76,10,2023-01-31,13
32516,76,14,2023-01-31,1
32517,76,62,2023-01-31,4


In [37]:
线路运输能力df = 线路运输能力df.set_index(['场地1', '场地2'])
线路运输能力df


货量
场地1 场地2      
1   8    2190
2   3    4756
    4    7252
    5    3797
    8    1604
...       ...
76  8    2920
    10   2455
    14   1175
    62   1429
77  6    1460

[1049 rows x 1 columns]

In [38]:
# 与DC5相关的线路
向dc5发货的场地 = rawData[(rawData['场地2'] == 5)]['场地1']
print(向dc5发货的场地)
print(f'发送货物到DC5的路线数量：{len(向dc5发货的场地)}')
可用的分发线路 = rawData[rawData['场地1'].isin(向dc5发货的场地)]
可用的分发线路

3        2
10       3
27       4
79       7
93       8
        ..
999     71
1011    72
1020    73
1035    74
1043    76
Name: 场地1, Length: 67, dtype: int64
发送货物到DC5的路线数量：67


,场地1,场地2,2021-01-01,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-08,...,2023-01-22,2023-01-23,2023-01-24,2023-01-25,2023-01-26,2023-01-27,2023-01-28,2023-01-29,2023-01-30,2023-01-31
1,2,3,9,9,9,9,9,9,9,9,...,3,4,4,4,4,4,4,4,4,4
2,2,4,2,2,2,2,2,2,2,2,...,4,4,5,5,5,5,5,5,5,5
3,2,5,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,5
4,2,8,2,2,2,2,2,2,2,2,...,4,4,4,4,4,4,4,4,4,4
5,2,9,33,33,33,33,33,33,33,33,...,5,5,5,5,5,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1043,76,5,3,3,3,3,3,3,3,3,...,4,5,5,5,6,6,6,7,7,8
1044,76,8,4,4,4,4,4,4,4,4,...,11,11,12,12,13,13,14,14,15,15
1045,76,10,3,3,3,3,3,3,3,3,...,9,10,10,11,11,12,12,13,13,13
1046,76,14,2,2,2,2,2,2,2,2,...,1,1,1,1,1,1,1,1,1,1


In [39]:
可用的分发线路tuple = list(map(tuple, 可用的分发线路.iloc[:, :2].values.tolist()))
可用的分发线路tuple

[(2, 3),
 (2, 4),
 (2, 5),
 (2, 8),
 (2, 9),
 (2, 10),
 (2, 14),
 (2, 62),
 (3, 4),
 (3, 5),
 (3, 8),
 (3, 9),
 (3, 10),
 (3, 12),
 (3, 14),
 (3, 19),
 (3, 21),
 (3, 22),
 (3, 23),
 (3, 28),
 (3, 30),
 (3, 35),
 (3, 36),
 (3, 38),
 (3, 40),
 (4, 3),
 (4, 5),
 (4, 8),
 (4, 9),
 (4, 10),
 (4, 14),
 (4, 17),
 (4, 19),
 (4, 20),
 (4, 21),
 (4, 22),
 (4, 23),
 (4, 30),
 (4, 34),
 (4, 35),
 (4, 36),
 (4, 38),
 (4, 40),
 (4, 62),
 (4, 64),
 (4, 65),
 (4, 70),
 (7, 3),
 (7, 4),
 (7, 5),
 (7, 8),
 (7, 9),
 (7, 10),
 (7, 12),
 (7, 14),
 (7, 34),
 (7, 35),
 (7, 40),
 (7, 62),
 (7, 65),
 (7, 70),
 (8, 3),
 (8, 4),
 (8, 5),
 (8, 9),
 (8, 10),
 (8, 13),
 (8, 14),
 (8, 15),
 (8, 25),
 (8, 27),
 (8, 32),
 (8, 33),
 (8, 34),
 (8, 35),
 (8, 38),
 (8, 40),
 (8, 43),
 (8, 44),
 (8, 59),
 (8, 62),
 (8, 64),
 (8, 65),
 (8, 70),
 (8, 76),
 (8, 78),
 (9, 3),
 (9, 4),
 (9, 5),
 (9, 8),
 (9, 10),
 (9, 12),
 (9, 13),
 (9, 14),
 (9, 19),
 (9, 21),
 (9, 22),
 (9, 23),
 (9, 25),
 (9, 32),
 (9, 34),
 (9, 35),
 (9, 3

In [40]:
可用的分发线路2023年 = pd.merge(可用的分发线路.iloc[:, :2], 可用的分发线路.iloc[:, -31:], left_index=True,
                                right_index=True)
可用的分发线路2023年

,场地1,场地2,2023-01-01,2023-01-02,2023-01-03,2023-01-04,2023-01-05,2023-01-06,2023-01-07,2023-01-08,...,2023-01-22,2023-01-23,2023-01-24,2023-01-25,2023-01-26,2023-01-27,2023-01-28,2023-01-29,2023-01-30,2023-01-31
1,2,3,1,1,1,1,1,1,1,2,...,3,4,4,4,4,4,4,4,4,4
2,2,4,2,2,2,3,3,3,3,3,...,4,4,5,5,5,5,5,5,5,5
3,2,5,10,10,10,9,9,9,9,8,...,6,6,6,6,6,6,6,6,6,5
4,2,8,2,2,2,2,2,3,3,3,...,4,4,4,4,4,4,4,4,4,4
5,2,9,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1043,76,5,2,2,2,2,2,2,2,3,...,4,5,5,5,6,6,6,7,7,8
1044,76,8,4,4,4,5,5,5,5,5,...,11,11,12,12,13,13,14,14,15,15
1045,76,10,4,4,4,4,5,5,5,5,...,9,10,10,11,11,12,12,13,13,13
1046,76,14,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [41]:
# YijNdarr 假设未关停站点时的该线路的运输量
print(len(可用的分发线路tuple))
print(可用的分发线路2023年)
YijNdarr = pd.melt(可用的分发线路2023年, id_vars=['场地1', '场地2'], value_name='货量', var_name='日期')
YijNdarr = YijNdarr['货量'].values
print(YijNdarr.shape)
YijNdarr

998
      场地1  场地2  2023-01-01  2023-01-02  2023-01-03  2023-01-04  2023-01-05   
1       2    3           1           1           1           1           1  \
2       2    4           2           2           2           3           3   
3       2    5          10          10          10           9           9   
4       2    8           2           2           2           2           2   
5       2    9           5           5           5           5           5   
...   ...  ...         ...         ...         ...         ...         ...   
1043   76    5           2           2           2           2           2   
1044   76    8           4           4           4           5           5   
1045   76   10           4           4           4           4           5   
1046   76   14           1           1           1           1           1   
1047   76   62           1           1           1           2           2   

      2023-01-06  2023-01-07  2023-01-08  ...  2023-01-22  

array([ 1,  2, 10, ..., 13,  1,  4], dtype=int64)

In [42]:
# WijNdarr 线路运输能力上限
print(len(可用的分发线路tuple))
WijNdarr = 线路运输能力df[线路运输能力df.index.isin(可用的分发线路tuple)]
print(WijNdarr)
WijNdarr = WijNdarr.values
print(WijNdarr.shape)
WijNdarr

998
           货量
场地1 场地2      
2   3    4756
    4    7252
    5    3797
    8    1604
    9    5394
...       ...
76  5    1241
    8    2920
    10   2455
    14   1175
    62   1429

[998 rows x 1 columns]
(998, 1)


array([[    4756],
       [    7252],
       [    3797],
       [    1604],
       [    5394],
       [    3910],
       [    5806],
       [    7354],
       [    2769],
       [    7479],
       [    2134],
       [    2163],
       [  143974],
       [   13827],
       [    4695],
       [    1460],
       [    1439],
       [    7455],
       [    3435],
       [    1460],
       [     730],
       [   10173],
       [    1944],
       [    8030],
       [    9032],
       [  129005],
       [  198371],
       [   99094],
       [    3427],
       [  205389],
       [  204534],
       [    5144],
       [    3249],
       [    2338],
       [    3080],
       [    3431],
       [  861093],
       [    4108],
       [    1790],
       [    5417],
       [    2580],
       [    1928],
       [    1460],
       [  163062],
       [    9490],
       [     730],
       [    5654],
       [  130145],
       [   48985],
       [  181336],
       [  768320],
       [  607742],
       [  44

In [43]:
场地处理能力df.loc[5]

货量    50764637
Name: 5, dtype: int64

In [44]:
周期 = 31


# 定义问题类
class MyProblem(ea.Problem):
    def __init__(self, PoolType='Thread'):
        name = 'MyProblem'
        M = 1  # 目标维数
        maxormins = [-1] * M  # 初始化maxormins（目标最小最大化标记列表，1：最小化该目标；-1：最大化该目标）

        Dim = len(可用的分发线路) * 周期  # 决策变量维数 shape(31, 998)
        varTypes = [1] * Dim  # 决策变量类型（0为实数，1为整数）
        lb = [0] * Dim  # 决策变量下界
        # ub = [场地处理能力df.loc[5][0]] * Dim  # 决策变量上界
        ub = [10] * Dim  # 决策变量上界
        lbin = [1] * Dim  # 决策变量是否包含下界
        ubin = [1] * Dim  # 决策变量是否包含上界

        # 调用父类构造方法
        super().__init__(name, M, maxormins, Dim, varTypes, lb, ub, lbin, ubin)

        # 设置用多线程还是多进程
        self.PoolType = PoolType
        if self.PoolType == 'Thread':
            self.pool = ThreadPool(mp.cpu_count() * 10)  # 设置池的大小
        elif self.PoolType == 'Process':
            self.pool = ProcessPool(mp.cpu_count())  # 设置池的大小

    # 定义evalVar方法计算目标函数值
    def evalVars(self, Vars):
        N = Vars.shape[0]
        args = list(zip(Vars, list(range(N))))
        resultList = list(self.pool.map(self.subVars, args))
        fList = [i[0] for i in resultList]
        CVList = [i[1] for i in resultList]
        f, CV = [np.array(fList), np.array(CVList)]
        return f, CV

    def subVars(self, args):
        Vars, idx = args
        x = Vars.copy()  # 决策变量矩阵

        # WijNdarr 线路运输能力上限
        # YijNdarr 假设未关停站点时的该线路的运输量
        # x 关闭站点时，线路运输的增量
        # print(np.repeat(WijNdarr, 周期).shape, (x + YijNdarr).shape)
        Wij = np.repeat(WijNdarr, 周期)
        Yij = YijNdarr
        y1 = np.power(Wij - (x + Yij), 2)
        y1 = np.log(y1)
        y2 = np.where(x == 0, 0, 1)

        y1 = np.reshape(y1, (周期, len(可用的分发线路)))
        y2 = np.reshape(y2, (周期, len(可用的分发线路)))

        y1 = np.sum(np.sum(y1, axis=1), axis=0)
        y2 = np.sum(np.sum(y2, axis=1), axis=0)

        y = y1 + y2

        '''
        s.t.
        '''
        CV = x + Yij - Wij
        CV = np.sort(CV, kind='quicksort', order=None)[::-1]
        return np.asarray([y]), np.asarray([CV[0]])



In [45]:
# 实例化问题对象
problem = MyProblem()
# 构建算法
algorithm = ea.soea_SGA_templet(
    problem,
    ea.Population(Encoding='BG',  # 种群的染色体是'BG'编码
                  NIND=100),  # 种群数量
    MAXGEN=9999,  # 最大进化代数
    logTras=1,  # 表示每隔多少代记录一次日志信息，0表示不记录。
    trappedValue=1e-6,  # 单目标优化陷入停滞的判断阈值。
    maxTrappedCount=1000  # 进化停滞计数器最大上限值。
)  # 表示每隔多少代记录一次日志信息，0表示不记录。
# algorithm.mutOper.Pm = 0.2  # 修改变异算子的变异概率
# algorithm.recOper.XOVR = 0.7  # 修改交叉算子的交叉概率

# 求解
res = ea.optimize(algorithm,
                  seed=1,
                  # prophet=prophet,
                  verbose=True,
                  drawing=1,
                  outputMsg=True,
                  drawLog=True,
                  saveFlag=True,
                  dirName='result_blue',
                  )



KeyboardInterrupt

